# Delayed RUL Estimation

Typical data-driven prognostics approaches assume that the RUL can be estimated using only the past data. However, model-based approaches typically consider the future as well. How does the performance differ if we consider future data in a data-driven approach? To answer this, we perform delayed RUL estimation, where we try to estimate the RUL $\frac{t-k}{2}$ ago where $t$ is the current time and $k$ is the length of historical data we use (hyperparameter). Effectively, we are considering a window of future operation along with past operation. We do so on the first N-CMAPSS dataset.

**For code on the performance without delayed RUL estimation, please see one of the other notebooks (such as `Missing Data Gap.ipynb`)**

In [8]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
torch.multiprocessing.set_sharing_strategy('file_system')
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from pickle import load
import h5py

import DataUtils
import Masking

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Modifying the Dataloader

The true label now needs to be a delayed $y$

In [85]:
class DelayedTrainNCMAPSSDataset(Dataset):
    def __init__(self, ds_no, timesteps=10):
        self.ds_no = ds_no
        self.fileloc = self.get_fileloc(ds_no)
        self.timesteps = timesteps
        self.scaler = self.get_scaler(ds_no)
    
    def __getitem__(self, index):
        index += int(self.timesteps/2)
        start = index - self.timesteps + 1
        if start < 0:
            start = 0
        
        indices = list(range(start, index+1))
        
        X_train, unit, A_dev, Y_dev = self.get_data(indices, index)
        
        X_train = self.scaler.transform(X_train)
        X_train[:,42] = A_dev[:,0]
        n_pad = self.timesteps - X_train.shape[0]
        X_train = np.pad(X_train, ((n_pad, 0),(0,0)), mode='constant')
        
        for i, row in enumerate(X_train): # TODO: implement the check for the y here too (done?)
            curr_unit = row[42] # ensure column 42 contains the unit number
            if curr_unit != unit and not np.all(row==0):
                X_train[i] = np.zeros_like(row)
                
        if  index - int(self.timesteps/2) < 0:
            return X_train, np.array([-1])
        
        return X_train, Y_dev
    
    def get_data(self, indices, index):
        with h5py.File(self.fileloc, 'r') as hdf:
            X_train = np.array(hdf.get('X_train')[indices])
            Y_dev = np.array(hdf.get('y_train')[index-int(self.timesteps/2)]) # actually want to do index - a value TODO, but need to check to make sure we don't grab data from a previous unit
            A_dev = X_train[:, -4:]
            
            unit = A_dev[-int(self.timesteps/2)+1, 0]
        
        return X_train, unit, A_dev, Y_dev
    
    def __len__(self):
        lengths = {
            1: 490663,
            2: 526344,
            3: 557127,
            4: 637745,
            5: 435060,
            6: 425720,
            7: 435017,
            8: 488538,
            9: 429991
        }
        
        return lengths[self.ds_no] - int(self.timesteps/2)
    
    def get_fileloc(self, ds_no):
        locations = {
            1: '/data/courseac/N-CMAPSS/subsampled/ds1.h5',
            2: '/data/courseac/N-CMAPSS/subsampled/ds2.h5',
            3: '/data/courseac/N-CMAPSS/subsampled/ds3.h5',
            4: '/data/courseac/N-CMAPSS/subsampled/ds4.h5',
            5: '/data/courseac/N-CMAPSS/subsampled/ds5.h5',
            6: '/data/courseac/N-CMAPSS/subsampled/ds6.h5',
            7: '/data/courseac/N-CMAPSS/subsampled/ds7.h5',
            8: '/data/courseac/N-CMAPSS/subsampled/ds8a.h5',
            9: '/data/courseac/N-CMAPSS/subsampled/ds8c.h5',
        }
        
        return locations[ds_no]
    
    def get_scaler(self, ds_no):
        return load(open('./scalers/scaler' + str(ds_no) + '.pkl', 'rb'))
    
class DelayedTestNCMAPSSDataset(DelayedTrainNCMAPSSDataset):
    def __init__(self, ds_no, timesteps=10):
        super().__init__(ds_no,timesteps)
    
    def get_data(self, indices, index):
        with h5py.File(self.fileloc, 'r') as hdf:
            X_test = np.array(hdf.get('X_test')[indices])
            Y_test = np.array(hdf.get('y_test')[index-int(self.timesteps/2)])
            A_test = X_test[:, -4:]
            
            unit = A_test[-int(self.timesteps/2), 0]
        
        return X_test, unit, A_test, Y_test
    
    def __len__(self):
        lengths = {
            1: 273523,
            2: 125374,
            3: 425156,
            4: 360256,
            5: 256204,
            6: 252244,
            7: 286978,
            8: 372299,
            9: 211781
        }
        
        return lengths[self.ds_no] - int(self.timesteps/2)

In [112]:
workers = 8
n_timesteps = 10
batch = 64
ds_no = 1

traindata = DelayedTrainNCMAPSSDataset(ds_no, timesteps=n_timesteps)
trainloader = DataLoader(traindata, batch_size=batch, shuffle=True, num_workers=workers)

testdata = DelayedTestNCMAPSSDataset(ds_no, timesteps=n_timesteps)
testloader = DataLoader(testdata, batch_size=batch, shuffle=False, num_workers=workers)

In [107]:
for i, (X, y) in enumerate(tqdm(trainloader)): 
    print(X[-1,:,-1])

  0%|▍                                                                                                                                                                            | 17/7666 [00:00<03:14, 39.35it/s]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.])
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1.

  1%|█                                                                                                                                                                            | 48/7666 [00:00<01:31, 83.68it/s]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.

  1%|█▎                                                                                                                                                                           | 60/7666 [00:00<01:28, 86.19it/s]

...

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.])
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
      

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌| 7648/7666 [01:10<00:00, 109.93it/s]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7666/7666 [01:10<00:00, 108.22it/s]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.])
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1.

## Defining an RUL Transformer Model

In [109]:
class RULTransformer(nn.Module):
    def __init__(self, timesteps):
        super().__init__()
        
        self.prediction_window = 1 # only estimates the next point
        self.input_len = timesteps # timesteps (half past, half future)
        self.n_cols = 35 # 35 columns reduced
        self.embed_dim = 64 # project to a 64-dimensional space
        
        self.input_projection = nn.Linear(self.input_len*self.n_cols, self.embed_dim) 
        self.positional_embed = nn.Parameter(torch.randn(self.embed_dim)) # learned positional embeddings
        
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=self.embed_dim, nhead=4, activation="gelu", dim_feedforward=64)
        self.transformer_blocks = nn.TransformerEncoder(self.encoder_layer, num_layers=1)
        
        self.rul_head = nn.Sequential(nn.Linear(self.embed_dim, self.prediction_window)) # linear rul prediction head 
    
    def forward(self, x):
        x = x.flatten(1)
        z = self.input_projection(x)
        z = z + self.positional_embed
        z = self.transformer_blocks(z)
        z = self.rul_head(z)
        
        return z.squeeze(1)

## Training on Clean Data

In [113]:
columns_kept = [True, True, True, True,
               True, True, True, True, True, True, True, True, True, True, True, True, True, True,
               True, True, True, True, True, True, True, True, True, True, True, True, True, True,
               False, False, False, False, False, False, False, False, False, False,
               True, True, True, False]

objective = nn.MSELoss()

rul_tran = RULTransformer(n_timesteps).to(device)

lr = 1e-4
n_epochs = 5
optim = torch.optim.Adam(rul_tran.parameters(), lr=lr)
losses = []

for n in range(n_epochs):
    counter = 0
    for i, (X, y) in enumerate(tqdm(trainloader)):
        X = X.to(device)[:,:,columns_kept]
        y = y.to(device)
        optim.zero_grad()
        yhat = rul_tran(X.float())
        loss = objective(yhat, y.float().squeeze(1))
        loss.backward()
        losses.append(loss.cpu().detach().numpy())
        optim.step()
        counter += 1
        
    print("Epoch:", n+1, "Loss:",np.mean(losses[-counter:][0]))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7667/7667 [01:07<00:00, 113.93it/s]


Epoch: 1 Loss: 2759.998


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7667/7667 [01:05<00:00, 117.28it/s]


Epoch: 2 Loss: 9.8431835


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7667/7667 [01:05<00:00, 117.28it/s]


Epoch: 3 Loss: 0.6759367


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7667/7667 [01:05<00:00, 116.88it/s]


Epoch: 4 Loss: 0.7006805


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7667/7667 [01:05<00:00, 116.61it/s]

Epoch: 5 Loss: 0.2582636


In [114]:
test_mses = []
yhats = []
ys = []
with torch.no_grad():
    for i, (X, y) in enumerate(tqdm(testloader)):
        yhat = rul_tran(X.float().to(device)[:,:,columns_kept])
        yhats.append(yhat.cpu())
        ys.append(y.cpu().unsqueeze(1))
        test_mse = objective(yhat, y.float().to(device))
        test_mses.append(test_mse.item())
print("Test MSE:", np.mean(test_mses))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4274/4274 [00:36<00:00, 117.09it/s]

Test MSE: 14.66226966785782


In [115]:
print("Test RMSE: {}".format(np.sqrt(np.mean(test_mses))))

Test RMSE: 3.8291343235590234


As a preliminary result, this looks promising. Let's vary the length of the timestep now and see what happens.

## Timestep Sweep

In [125]:
timestep_grid = [2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60, 70, 80, 90, 100]

In [126]:
def train_model(timestep, trainloader):
    columns_kept = [True, True, True, True,
               True, True, True, True, True, True, True, True, True, True, True, True, True, True,
               True, True, True, True, True, True, True, True, True, True, True, True, True, True,
               False, False, False, False, False, False, False, False, False, False,
               True, True, True, False]

    objective = nn.MSELoss()

    rul_tran = RULTransformer(timestep).to(device)

    lr = 1e-4
    n_epochs = 5
    optim = torch.optim.Adam(rul_tran.parameters(), lr=lr)
    losses = []

    for n in range(n_epochs):
        counter = 0
        for i, (X, y) in enumerate(tqdm(trainloader)):
            X = X.to(device)[:,:,columns_kept]
            y = y.to(device)
            optim.zero_grad()
            yhat = rul_tran(X.float())
            loss = objective(yhat, y.float().squeeze(1))
            loss.backward()
            losses.append(loss.cpu().detach().numpy())
            optim.step()
            counter += 1

        print("Epoch:", n+1, "Loss:",np.mean(losses[-counter:][0]))
        
    return rul_tran

def validate_model(model, testloader):
    test_mses = []
    yhats = []
    ys = []
    with torch.no_grad():
        for i, (X, y) in enumerate(tqdm(testloader)):
            yhat = model(X.float().to(device)[:,:,columns_kept])
            yhats.append(yhat.cpu())
            ys.append(y.cpu().unsqueeze(1))
            test_mse = objective(yhat, y.float().to(device))
            test_mses.append(test_mse.item())
    print("Test MSE:", np.mean(test_mses))
    print("Test RMSE:", np.sqrt(np.mean(test_mses)))
    
    return np.sqrt(np.mean(test_mses))

In [ ]:
rmses = {}
for t in timestep_grid:
    workers = 8
    batch = 64
    ds_no = 1

    traindata = DelayedTrainNCMAPSSDataset(ds_no, timesteps=t)
    trainloader = DataLoader(traindata, batch_size=batch, shuffle=True, num_workers=workers)

    testdata = DelayedTestNCMAPSSDataset(ds_no, timesteps=t)
    testloader = DataLoader(testdata, batch_size=batch, shuffle=False, num_workers=workers)
    
    rul_model = train_model(t, trainloader)
    rmse = validate_model(rul_model, testloader)
    
    rmses[str(t)] = rmse
    print(rmses)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7667/7667 [01:00<00:00, 126.72it/s]


Epoch: 1 Loss: 2644.0146


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7667/7667 [01:00<00:00, 127.01it/s]


Epoch: 2 Loss: 41.315365


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7667/7667 [01:00<00:00, 126.92it/s]


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7667/7667 [01:04<00:00, 118.29it/s]


Epoch: 2 Loss: 11.793373


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7667/7667 [01:05<00:00, 117.38it/s]


Epoch: 3 Loss: 0.866212


 89%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                  | 6846/7667 [00:58<00:07, 115.69it/s]